In [7]:
import pandas as pd
import numpy as np
import re, os, zipfile
from datetime import datetime

input_folder = "/kaggle/input/ba-dataset/"  
output_path = "/kaggle/working/BA_dataset_cleaned.csv"

print("Files in input folder:")
print(os.listdir(input_folder))

for file in os.listdir(input_folder):
    if file.endswith(".zip"):
        with zipfile.ZipFile(os.path.join(input_folder, file), 'r') as zip_ref:
            zip_ref.extractall(input_folder)
        print(f"Extracted {file}")

Files in input folder:
['Review_8_Final.xlsx', 'Review_3_Final.xlsx', 'Review_15_final.xlsx', 'Review_9_Final.xlsx', 'Review_18_final.xlsx', 'Review_5_Final.xlsx', 'Review_11_final.xlsx', 'Review_10_final.xlsx', 'Review_7_Final.xlsx', 'Review_19_final.xlsx', 'Review_16_final.xlsx', 'Review_6_Final.xlsx', 'Review_2_Final.xlsx', 'Review_4 _Final.xlsx', 'Review_13_final.xlsx', 'Review_20_final.xlsx', 'Review_1_Final.xlsx', 'Review_17_final.xlsx', 'Review_14_final.xlsx', 'Review_12_final.xlsx']


In [8]:

# -------------------------------
# Preprocessing Function
# -------------------------------
def preprocess_dataset(df, source_name):
    df['source_file'] = source_name

    # Handle missing values
    missing_ratio = df.isnull().mean()
    df.drop(columns=missing_ratio[missing_ratio > 0.5].index, inplace=True)
    df = df.dropna(thresh=df.shape[1] * 0.5)

    # Impute numeric missing with median
    for col in df.select_dtypes(include=[np.number]).columns:
        if df[col].isnull().sum() > 0:
            df[col].fillna(df[col].median(), inplace=True)

    # Drop duplicates
    df.drop_duplicates(inplace=True)

    # Drop unnecessary columns
    drop_cols = ['Brand Entry Time into China', 'Comment Interval Time']
    df.drop(columns=[c for c in drop_cols if c in df.columns], inplace=True, errors='ignore')

    # Remove duplicate columns
    dup_cols = ['performance review.1', 'interior review.1']
    df.drop(columns=[c for c in dup_cols if c in df.columns], inplace=True, errors='ignore')

    # Standardize column names
    df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('.', '', regex=False)

    # Combine Year + Month → Date (robust version)
    year_col = next((c for c in df.columns if 'year' in c and ('purchase' in c or 'buy' in c or 'sale' in c)), None)
    month_col = next((c for c in df.columns if 'month' in c and ('purchase' in c or 'buy' in c or 'sale' in c)), None)

    if not month_col:
        month_col = next((c for c in df.columns if c == 'month' or c.endswith('_month')), None)
    if not year_col:
        year_col = next((c for c in df.columns if c == 'year' or c.endswith('_year')), None)

    print(f"[{source_name}] Month column: {month_col}, Year column: {year_col}")

    if year_col and month_col:
        df[year_col] = pd.to_numeric(df[year_col], errors='coerce')
        df[month_col] = pd.to_numeric(df[month_col], errors='coerce')
        df.loc[~df[month_col].between(1, 12), month_col] = np.nan

        df['purchase_date'] = pd.to_datetime(
            {'year': df[year_col], 'month': df[month_col], 'day': 1},
            errors='coerce'
        )
    else:
        df['purchase_date'] = pd.NaT

    # Fix average fuel consumption
    if 'average_fuel_consumption' in df.columns:
        def extract_number(x):
            if isinstance(x, str):
                match = re.search(r'\d+\.?\d*', x)
                return float(match.group()) if match else np.nan
            return x
        df['average_fuel_consumption'] = df['average_fuel_consumption'].apply(extract_number)

    # Convert numeric columns
    for col in ['official_guide_price', 'ex-factory_price', 'sales']:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')

    # Text preprocessing
    text_cols = [
        'advantage', 'disadvantage', 'appearance_review', 'interior_review',
        'features_evaluation', 'performance_review', 'energy_consumption_evaluation',
        'comfort_evaluation'
    ]
    text_cols = [c for c in text_cols if c in df.columns]

    if text_cols:
        df['combined_reviews'] = df[text_cols].astype(str).agg(' '.join, axis=1)
    else:
        df['combined_reviews'] = ""

    stop_words = set([
        'a','an','the','and','or','but','if','in','on','of','for','to','with','at','by','from',
        'this','that','is','are','was','were','be','been','it','its','as','so','we','you','i'
    ])

    def clean_text(text):
        text = text.lower()
        text = re.sub(r'[\U00010000-\U0010ffff]', '', text)  # remove emojis
        text = re.sub(r'[^a-z\s]', ' ', text)
        text = ' '.join([w for w in text.split() if w not in stop_words])
        text = re.sub(r'\s+', ' ', text).strip()
        return text

    df['cleaned_text'] = df['combined_reviews'].apply(clean_text)

    # Feature engineering
    if all(col in df.columns for col in ['official_guide_price', 'ex-factory_price']):
        df['price_difference'] = df['official_guide_price'] - df['ex-factory_price']

    if 'brand_establishment_date' in df.columns:
        df['years_since_brand_establishment'] = 2025 - df['brand_establishment_date']

    if 'brand_entry_time_into_china' in df.columns:
        df['years_in_china'] = 2025 - df['brand_entry_time_into_china']
    else:
        df['years_in_china'] = np.nan

    if 'sales' in df.columns:
        df = df[df['sales'].notnull()]

    return df

# -------------------------------
# Combine all datasets
# -------------------------------
combined_df = pd.DataFrame()

for file in os.listdir(input_folder):
    if file.endswith((".xlsx", ".csv")):
        try:
            path = os.path.join(input_folder, file)
            df = pd.read_excel(path) if file.endswith(".xlsx") else pd.read_csv(path)
            print(f"Processing: {file}")
            cleaned_df = preprocess_dataset(df, file)
            combined_df = pd.concat([combined_df, cleaned_df], ignore_index=True)
        except Exception as e:
            print(f"Error processing {file}: {e}")

print("All datasets combined successfully.")
print("Final combined shape:", combined_df.shape)

# -------------------------------
# Export
# -------------------------------
combined_df.to_csv(output_path, index=False)
print(f"Combined dataset saved to: {output_path}")


Processing: Review_8_Final.xlsx


/tmp/ipykernel_48/2861638746.py:15: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(df[col].median(), inplace=True)


[Review_8_Final.xlsx] Month column: month_of_purchase, Year column: year_of_purchase
Processing: Review_3_Final.xlsx
[Review_3_Final.xlsx] Month column: month_of_purchase, Year column: year_of_purchase


/tmp/ipykernel_48/2861638746.py:15: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(df[col].median(), inplace=True)


Processing: Review_15_final.xlsx
[Review_15_final.xlsx] Month column: month_of_purchase, Year column: year_of_purchase


...

Processing: Review_14_final.xlsx
[Review_14_final.xlsx] Month column: month_of_purchase, Year column: year_of_purchase


/tmp/ipykernel_48/2861638746.py:15: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(df[col].median(), inplace=True)


Processing: Review_12_final.xlsx
[Review_12_final.xlsx] Month column: month_of_purchase, Year column: year_of_purchase


/tmp/ipykernel_48/2861638746.py:15: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(df[col].median(), inplace=True)


All datasets combined successfully.
Final combined shape: (217292, 50)
Combined dataset saved to: /kaggle/working/BA_dataset_cleaned.csv
